In [1]:
# Data visualization
import numpy as np
import pandas as pd 

# Keras
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.optimizers import Adam
from keras.utils.np_utils import to_categorical
from keras.callbacks import EarlyStopping, TensorBoard
import keras_tuner as kt

# Train-Test
from sklearn.model_selection import train_test_split
# Classification Report
from sklearn.metrics import confusion_matrix, precision_recall_fscore_support

import pickle

import warnings
warnings.filterwarnings('ignore')

## 1. Important Landmarks and Important functions

In [2]:
# Determine important landmarks for plank
IMPORTANT_LMS = [
    "NOSE",
    "LEFT_SHOULDER",
    "RIGHT_SHOULDER",
    "RIGHT_ELBOW",
    "LEFT_ELBOW",
    "RIGHT_WRIST",
    "LEFT_WRIST",
    "LEFT_HIP",
    "RIGHT_HIP",
]

# Generate all columns of the data frame

HEADERS = ["label"] # Label column

for lm in IMPORTANT_LMS:
    HEADERS += [f"{lm.lower()}_x", f"{lm.lower()}_y", f"{lm.lower()}_z", f"{lm.lower()}_v"]

In [12]:
def describe_dataset(dataset_path: str):
    '''
    Describe dataset
    '''

    data = pd.read_csv(dataset_path)
    print(f"Headers: {list(data.columns.values)}")
    print(f'Number of rows: {data.shape[0]} \nNumber of columns: {data.shape[1]}\n')
    print(f"Labels: \n{data['label'].value_counts()}\n")
    print(f"Missing values: {data.isnull().values.any()}\n")
    
    duplicate = data[data.duplicated()]
    print(f"Duplicate Rows : {len(duplicate.sum(axis=1))}")

    return data


# Remove duplicate rows (optional)
def remove_duplicate_rows(dataset_path: str):
    '''
    Remove duplicated data from the dataset then save it to another files
    '''
    
    df = pd.read_csv(dataset_path)
    df.drop_duplicates(keep="first", inplace=True)
    df.to_csv(f"cleaned_train.csv", sep=',', encoding='utf-8', index=False)


def round_up_metric_results(results) -> list:
    '''Round up metrics results such as precision score, recall score, ...'''
    return list(map(lambda el: round(el, 3), results))

## 2. Describe Dataset & Split Data

In [4]:
# load dataset
df = describe_dataset("./train.csv")

# Categorizing label
df.loc[df["label"] == "C", "label"] = 0
df.loc[df["label"] == "L", "label"] = 1

Headers: ['label', 'nose_x', 'nose_y', 'nose_z', 'nose_v', 'left_shoulder_x', 'left_shoulder_y', 'left_shoulder_z', 'left_shoulder_v', 'right_shoulder_x', 'right_shoulder_y', 'right_shoulder_z', 'right_shoulder_v', 'right_elbow_x', 'right_elbow_y', 'right_elbow_z', 'right_elbow_v', 'left_elbow_x', 'left_elbow_y', 'left_elbow_z', 'left_elbow_v', 'right_wrist_x', 'right_wrist_y', 'right_wrist_z', 'right_wrist_v', 'left_wrist_x', 'left_wrist_y', 'left_wrist_z', 'left_wrist_v', 'left_hip_x', 'left_hip_y', 'left_hip_z', 'left_hip_v', 'right_hip_x', 'right_hip_y', 'right_hip_z', 'right_hip_v']
Number of rows: 15372 
Number of columns: 37

Labels: 
C    8238
L    7134
Name: label, dtype: int64

Missing values: False

Duplicate Rows : 0


In [5]:
with open("./model/input_scaler.pkl", "rb") as f:
    sc = pickle.load(f)

In [6]:
# Standard Scaling of features
x = df.drop("label", axis = 1)
x = pd.DataFrame(sc.transform(x))

y = df["label"]

# # Converting prediction to categorical
y_cat = to_categorical(y)

In [7]:
x_train, x_test, y_train, y_test = train_test_split(x.values, y_cat, test_size=0.2, random_state=1234)

## 3. Build Model

### 3.1. Set up

In [9]:
stop_early = EarlyStopping(monitor='val_loss', patience=3)

# Final Results
final_models = {}

In [8]:
def describe_model(model):
    '''
    Describe Model architecture
    '''
    print(f"Describe models architecture")
    for i, layer in enumerate(model.layers):
        number_of_units = layer.units if hasattr(layer, 'units') else 0

        if hasattr(layer, "activation"):
            print(f"Layer-{i + 1}: {number_of_units} units, func: ", layer.activation)
        else:
            print(f"Layer-{i + 1}: {number_of_units} units, func: None")


def get_best_model(tuner):
    '''
    Describe and return the best model found from keras tuner
    '''
    best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]
    best_model = tuner.hypermodel.build(best_hps)

    describe_model(best_model)

    print("\nOther params:")
    ignore_params = ["tuner", "activation", "layer", "epoch"]
    for param, value in best_hps.values.items():
        if not any(word in param for word in ignore_params):
            print(f"{param}: {value}")

    return best_model

### 3.2. Model with 3 layers

In [12]:
def model_3l_builder(hp):
    model = Sequential()
    model.add(Dense(36, input_dim = 36, activation = "relu"))

    hp_activation = hp.Choice('activation', values=['relu', 'tanh'])
    hp_layer_1 = hp.Int('layer_1', min_value=32, max_value=512, step=32)
    hp_learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])

    model.add(Dense(units=hp_layer_1, activation=hp_activation))
    model.add(Dense(2, activation = "softmax"))

    model.compile(optimizer=Adam(learning_rate=hp_learning_rate), loss="categorical_crossentropy", metrics = ["accuracy"])
    
    return model

In [17]:
tuner_3l = kt.Hyperband(
    model_3l_builder,
    objective='val_accuracy',
    max_epochs=10,
    directory='keras_tuner_dir',
    project_name='keras_tuner_demo',
)
tuner_3l.search(x_train, y_train, validation_data=(x_test, y_test), epochs=10, callbacks=[stop_early])

Trial 30 Complete [00h 00m 46s]
val_accuracy: 0.9980487823486328

Best val_accuracy So Far: 0.9980487823486328
Total elapsed time: 00h 08m 25s
INFO:tensorflow:Oracle triggered exit


In [19]:
model_3l = get_best_model(tuner_3l)
model_3l.fit(x_train, y_train, epochs=100, batch_size=10, validation_data=(x_test, y_test), callbacks=[stop_early])

Describe models architecture
Layer-1: 36 units, func:  <function relu at 0x16bf85b80>
Layer-2: 448 units, func:  <function tanh at 0x16bf85ee0>
Layer-3: 2 units, func:  <function softmax at 0x16bf85160>

Other params:
learning_rate: 0.001
Epoch 1/100
   5/1230 [..............................] - ETA: 19s - loss: 0.6247 - accuracy: 0.6600 

2022-11-23 09:54:28.588878: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


1230/1230 [==============================] - ETA: 0s - loss: 0.0504 - accuracy: 0.9848

2022-11-23 09:54:39.929268: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


1230/1230 [==============================] - 14s 11ms/step - loss: 0.0504 - accuracy: 0.9848 - val_loss: 0.0889 - val_accuracy: 0.9717
Epoch 2/100
1230/1230 [==============================] - 13s 11ms/step - loss: 0.0241 - accuracy: 0.9940 - val_loss: 0.0188 - val_accuracy: 0.9948
Epoch 3/100
1230/1230 [==============================] - 13s 10ms/step - loss: 0.0187 - accuracy: 0.9946 - val_loss: 0.0127 - val_accuracy: 0.9964
Epoch 4/100
1230/1230 [==============================] - 13s 10ms/step - loss: 0.0179 - accuracy: 0.9950 - val_loss: 0.0140 - val_accuracy: 0.9958
Epoch 5/100
1230/1230 [==============================] - 13s 11ms/step - loss: 0.0145 - accuracy: 0.9958 - val_loss: 0.0211 - val_accuracy: 0.9958
Epoch 6/100
1230/1230 [==============================] - 13s 11ms/step - loss: 0.0143 - accuracy: 0.9958 - val_loss: 0.0093 - val_accuracy: 0.9984
Epoch 7/100
1230/1230 [==============================] - 13s 11ms/step - loss: 0.0118 - accuracy: 0.9966 - val_loss: 0.0077 - val_

In [20]:
final_models["3_layers"] = model_3l

### 3.3. Model with 5 layers

In [21]:
def model_5l_builder(hp):
    model = Sequential()
    model.add(Dense(36, input_dim = 36, activation = "relu"))

    hp_activation = hp.Choice('activation', values=['relu', 'tanh'])
    hp_layer_1 = hp.Int('layer_1', min_value=32, max_value=512, step=32)
    hp_layer_2 = hp.Int('layer_2', min_value=32, max_value=512, step=32)
    hp_layer_3 = hp.Int('layer_3', min_value=32, max_value=512, step=32)
    hp_learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])

    model.add(Dense(units=hp_layer_1, activation=hp_activation))
    model.add(Dense(units=hp_layer_2, activation=hp_activation))
    model.add(Dense(units=hp_layer_3, activation=hp_activation))
    model.add(Dense(2, activation = "softmax"))

    model.compile(optimizer=Adam(learning_rate=hp_learning_rate), loss="categorical_crossentropy", metrics = ["accuracy"])
    
    return model

In [24]:
tuner_5l = kt.Hyperband(
    model_5l_builder,
    objective='val_accuracy',
    max_epochs=10,
    directory='keras_tuner_dir',
    project_name='keras_tuner_demo_2'
)
tuner_5l.search(x_train, y_train, validation_data=(x_test, y_test), epochs=10, callbacks=[stop_early, TensorBoard("./keras_tuner_dir/logs")])

Trial 30 Complete [00h 00m 54s]
val_accuracy: 0.9973983764648438

Best val_accuracy So Far: 0.9986991882324219
Total elapsed time: 00h 11m 12s
INFO:tensorflow:Oracle triggered exit


In [25]:
model_5l = get_best_model(tuner_5l)
model_5l.fit(x_train, y_train, epochs=100, batch_size=10, validation_data=(x_test, y_test), callbacks=[stop_early])

Describe models architecture
Layer-1: 36 units, func:  <function relu at 0x16bf85b80>
Layer-2: 160 units, func:  <function relu at 0x16bf85b80>
Layer-3: 352 units, func:  <function relu at 0x16bf85b80>
Layer-4: 64 units, func:  <function relu at 0x16bf85b80>
Layer-5: 2 units, func:  <function softmax at 0x16bf85160>

Other params:
learning_rate: 0.001
Epoch 1/100


2022-11-23 10:15:07.538823: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


1230/1230 [==============================] - ETA: 0s - loss: 0.0494 - accuracy: 0.9848

2022-11-23 10:15:21.397335: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


1230/1230 [==============================] - 16s 13ms/step - loss: 0.0494 - accuracy: 0.9848 - val_loss: 0.0152 - val_accuracy: 0.9958
Epoch 2/100
1230/1230 [==============================] - 16s 13ms/step - loss: 0.0238 - accuracy: 0.9932 - val_loss: 0.0145 - val_accuracy: 0.9954
Epoch 3/100
1230/1230 [==============================] - 16s 13ms/step - loss: 0.0193 - accuracy: 0.9947 - val_loss: 0.0146 - val_accuracy: 0.9971
Epoch 4/100
1230/1230 [==============================] - 16s 13ms/step - loss: 0.0169 - accuracy: 0.9950 - val_loss: 0.0140 - val_accuracy: 0.9964
Epoch 5/100
1230/1230 [==============================] - 15s 13ms/step - loss: 0.0160 - accuracy: 0.9960 - val_loss: 0.0154 - val_accuracy: 0.9964
Epoch 6/100
1230/1230 [==============================] - 16s 13ms/step - loss: 0.0135 - accuracy: 0.9963 - val_loss: 0.0126 - val_accuracy: 0.9961
Epoch 7/100
1230/1230 [==============================] - 16s 13ms/step - loss: 0.0126 - accuracy: 0.9960 - val_loss: 0.0098 - val_

In [26]:
final_models["5_layers"] = model_5l

### 3.4. Model with 7 layers include Dropout

In [27]:
def model_7lD_builder(hp):
    model = Sequential()
    model.add(Dense(36, input_dim = 36, activation = "relu"))

    hp_activation = hp.Choice('activation', values=['relu', 'tanh'])
    hp_layer_1 = hp.Int('layer_1', min_value=32, max_value=512, step=32)
    hp_layer_2 = hp.Int('layer_2', min_value=32, max_value=512, step=32)
    hp_layer_3 = hp.Int('layer_3', min_value=32, max_value=512, step=32)
    hp_dropout_1 = hp.Float('dropout_1', min_value=0.1, max_value=0.5, step=0.1)
    hp_dropout_2 = hp.Float('dropout_2', min_value=0.1, max_value=0.5, step=0.1)
    hp_learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])

    model.add(Dense(units=hp_layer_1, activation=hp_activation))
    model.add(Dropout(rate=hp_dropout_1))
    model.add(Dense(units=hp_layer_2, activation=hp_activation))
    model.add(Dropout(rate=hp_dropout_2))
    model.add(Dense(units=hp_layer_3, activation=hp_activation))
    model.add(Dense(2, activation = "softmax"))

    model.compile(optimizer=Adam(learning_rate=hp_learning_rate), loss="categorical_crossentropy", metrics = ["accuracy"])
    
    return model

In [28]:
tuner_7lD = kt.Hyperband(
    model_7lD_builder,
    objective='accuracy',
    max_epochs=10,
    directory='keras_tuner_dir',
    project_name='keras_tuner_demo_3'
)
tuner_7lD.search(x_train, y_train, validation_data=(x_test, y_test), epochs=10, callbacks=[stop_early, TensorBoard("./keras_tuner_dir/logs")])

Trial 30 Complete [00h 01m 04s]
accuracy: 0.9945515394210815

Best accuracy So Far: 0.9969098567962646
Total elapsed time: 00h 12m 19s
INFO:tensorflow:Oracle triggered exit


In [29]:
model_7lD = get_best_model(tuner_7lD)
model_7lD.fit(x_train, y_train, epochs=100, batch_size=10, validation_data=(x_test, y_test), callbacks=[stop_early])

Describe models architecture
Layer-1: 36 units, func:  <function relu at 0x16bf85b80>
Layer-2: 320 units, func:  <function relu at 0x16bf85b80>
Layer-3: 0 units, func: None
Layer-4: 96 units, func:  <function relu at 0x16bf85b80>
Layer-5: 0 units, func: None
Layer-6: 448 units, func:  <function relu at 0x16bf85b80>
Layer-7: 2 units, func:  <function softmax at 0x16bf85160>

Other params:
dropout_1: 0.30000000000000004
dropout_2: 0.30000000000000004
learning_rate: 0.001
Epoch 1/100


2022-11-23 10:37:14.947724: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


1230/1230 [==============================] - ETA: 0s - loss: 0.0592 - accuracy: 0.9811

2022-11-23 10:37:31.869492: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


1230/1230 [==============================] - 20s 16ms/step - loss: 0.0592 - accuracy: 0.9811 - val_loss: 0.0177 - val_accuracy: 0.9961
Epoch 2/100
1230/1230 [==============================] - 19s 16ms/step - loss: 0.0235 - accuracy: 0.9934 - val_loss: 0.0164 - val_accuracy: 0.9951
Epoch 3/100
1230/1230 [==============================] - 19s 15ms/step - loss: 0.0206 - accuracy: 0.9945 - val_loss: 0.0150 - val_accuracy: 0.9945
Epoch 4/100
1230/1230 [==============================] - 19s 15ms/step - loss: 0.0175 - accuracy: 0.9951 - val_loss: 0.0160 - val_accuracy: 0.9961
Epoch 5/100
1230/1230 [==============================] - 19s 15ms/step - loss: 0.0176 - accuracy: 0.9950 - val_loss: 0.0131 - val_accuracy: 0.9964
Epoch 6/100
1230/1230 [==============================] - 19s 15ms/step - loss: 0.0137 - accuracy: 0.9960 - val_loss: 0.0091 - val_accuracy: 0.9984
Epoch 7/100
1230/1230 [==============================] - 19s 15ms/step - loss: 0.0141 - accuracy: 0.9959 - val_loss: 0.0121 - val_

In [30]:
final_models["7_layers_with_dropout"] = model_7lD

### 3.5. Model with 7 layers

In [11]:
def model_7l_builder(hp):
    model = Sequential()
    model.add(Dense(36, input_dim = 36, activation = "relu"))

    hp_activation = hp.Choice('activation', values=['relu', 'tanh'])
    hp_layer_1 = hp.Int('layer_1', min_value=32, max_value=512, step=32)
    hp_layer_2 = hp.Int('layer_2', min_value=32, max_value=512, step=32)
    hp_layer_3 = hp.Int('layer_3', min_value=32, max_value=512, step=32)
    hp_layer_4 = hp.Int('layer_4', min_value=32, max_value=512, step=32)
    hp_layer_5 = hp.Int('layer_5', min_value=32, max_value=512, step=32)
    hp_learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])

    model.add(Dense(units=hp_layer_1, activation=hp_activation))
    model.add(Dense(units=hp_layer_2, activation=hp_activation))
    model.add(Dense(units=hp_layer_3, activation=hp_activation))
    model.add(Dense(units=hp_layer_4, activation=hp_activation))
    model.add(Dense(units=hp_layer_5, activation=hp_activation))
    model.add(Dense(2, activation = "softmax"))

    model.compile(optimizer=Adam(learning_rate=hp_learning_rate), loss="categorical_crossentropy", metrics = ["accuracy"])
    
    return model

In [12]:
tuner_7l = kt.Hyperband(
    model_7l_builder,
    objective='val_accuracy',
    max_epochs=10,
    directory='keras_tuner_dir',
    project_name='keras_tuner_demo_6'
)
tuner_7l.search(x_train, y_train, validation_data=(x_test, y_test), epochs=10, callbacks=[stop_early, TensorBoard("./keras_tuner_dir/logs")])

Trial 30 Complete [00h 00m 51s]
val_accuracy: 0.9973983764648438

Best val_accuracy So Far: 0.9977235794067383
Total elapsed time: 00h 02m 22s
INFO:tensorflow:Oracle triggered exit


In [13]:
model_7l = get_best_model(tuner_7l)
model_7l.fit(x_train, y_train, epochs=100, batch_size=10, validation_data=(x_test, y_test), callbacks=[stop_early])

Describe models architecture
Layer-1: 36 units, func:  <function relu at 0x13a058b80>
Layer-2: 192 units, func:  <function tanh at 0x13a058ee0>
Layer-3: 320 units, func:  <function tanh at 0x13a058ee0>
Layer-4: 448 units, func:  <function tanh at 0x13a058ee0>
Layer-5: 224 units, func:  <function tanh at 0x13a058ee0>
Layer-6: 448 units, func:  <function tanh at 0x13a058ee0>
Layer-7: 2 units, func:  <function softmax at 0x13a058160>

Other params:
learning_rate: 0.0001
Epoch 1/100
   1/1230 [..............................] - ETA: 8:04 - loss: 0.6561 - accuracy: 0.6000

2022-11-23 14:29:10.795739: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


1230/1230 [==============================] - ETA: 0s - loss: 0.0720 - accuracy: 0.9748

2022-11-23 14:29:24.355056: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


1230/1230 [==============================] - 16s 13ms/step - loss: 0.0720 - accuracy: 0.9748 - val_loss: 0.0219 - val_accuracy: 0.9941
Epoch 2/100
1230/1230 [==============================] - 15s 12ms/step - loss: 0.0319 - accuracy: 0.9910 - val_loss: 0.0353 - val_accuracy: 0.9893
Epoch 3/100
1230/1230 [==============================] - 15s 12ms/step - loss: 0.0262 - accuracy: 0.9927 - val_loss: 0.0149 - val_accuracy: 0.9958
Epoch 4/100
1230/1230 [==============================] - 15s 12ms/step - loss: 0.0221 - accuracy: 0.9936 - val_loss: 0.0125 - val_accuracy: 0.9964
Epoch 5/100
1230/1230 [==============================] - 15s 12ms/step - loss: 0.0195 - accuracy: 0.9943 - val_loss: 0.0171 - val_accuracy: 0.9951
Epoch 6/100
1230/1230 [==============================] - 16s 13ms/step - loss: 0.0186 - accuracy: 0.9942 - val_loss: 0.0128 - val_accuracy: 0.9971
Epoch 7/100
1230/1230 [==============================] - 16s 13ms/step - loss: 0.0170 - accuracy: 0.9951 - val_loss: 0.0132 - val_

In [16]:
final_models["7_layers"] = model_7l

### 3.6. Describe final models

In [10]:
for name, model in final_models.items():
    print(f"{name}: ", end="")
    describe_model(model)
    print()

3_layers: Describe models architecture
Layer-1: 36 units, func:  <function relu at 0x17fe29b80>
Layer-2: 448 units, func:  <function tanh at 0x17fe29ee0>
Layer-3: 2 units, func:  <function softmax at 0x17fe29160>

5_layers: Describe models architecture
Layer-1: 36 units, func:  <function relu at 0x17fe29b80>
Layer-2: 160 units, func:  <function relu at 0x17fe29b80>
Layer-3: 352 units, func:  <function relu at 0x17fe29b80>
Layer-4: 64 units, func:  <function relu at 0x17fe29b80>
Layer-5: 2 units, func:  <function softmax at 0x17fe29160>

7_layers_with_dropout: Describe models architecture
Layer-1: 36 units, func:  <function relu at 0x17fe29b80>
Layer-2: 320 units, func:  <function relu at 0x17fe29b80>
Layer-3: 0 units, func: None
Layer-4: 96 units, func:  <function relu at 0x17fe29b80>
Layer-5: 0 units, func: None
Layer-6: 448 units, func:  <function relu at 0x17fe29b80>
Layer-7: 2 units, func:  <function softmax at 0x17fe29160>

7_layers: Describe models architecture
Layer-1: 36 units,

## 4. Model Evaluation

### 4.1. Train set evaluation

In [13]:
train_set_results = []

for name, model in final_models.items():
    # Evaluate model
    predict_x = model.predict(x_test, verbose=False) 
    y_pred_class = np.argmax(predict_x, axis=1)
    y_test_class = np.argmax(y_test, axis=1)

    cm = confusion_matrix(y_test_class, y_pred_class, labels=[0, 1])
    (p_score, r_score, f_score, _) = precision_recall_fscore_support(y_test_class, y_pred_class, labels=[0, 1])
    
    train_set_results.append(( name, round_up_metric_results(p_score), round_up_metric_results(r_score), round_up_metric_results(f_score), cm ))

train_set_results.sort(key=lambda k: sum(k[3]), reverse=True)
pd.DataFrame(train_set_results, columns=["Model", "Precision Score", "Recall Score", "F1 score", "Confusion Matrix"])

,Model,Precision Score,Recall Score,F1 score,Confusion Matrix
0,7_layers,"[0.998, 0.997]","[0.998, 0.997]","[0.998, 0.997]","[[1675, 4], [4, 1392]]"
1,3_layers,"[0.997, 0.996]","[0.997, 0.996]","[0.997, 0.996]","[[1674, 5], [5, 1391]]"
2,7_layers_with_dropout,"[0.998, 0.995]","[0.996, 0.997]","[0.997, 0.996]","[[1672, 7], [4, 1392]]"
3,5_layers,"[0.996, 0.996]","[0.996, 0.996]","[0.996, 0.996]","[[1673, 6], [6, 1390]]"


### 4.2. Test set evaluation

In [14]:
# load dataset
test_df = describe_dataset("./test.csv")

# Categorizing label
test_df.loc[test_df["label"] == "C", "label"] = 0
test_df.loc[test_df["label"] == "L", "label"] = 1

Headers: ['label', 'nose_x', 'nose_y', 'nose_z', 'nose_v', 'left_shoulder_x', 'left_shoulder_y', 'left_shoulder_z', 'left_shoulder_v', 'right_shoulder_x', 'right_shoulder_y', 'right_shoulder_z', 'right_shoulder_v', 'right_elbow_x', 'right_elbow_y', 'right_elbow_z', 'right_elbow_v', 'left_elbow_x', 'left_elbow_y', 'left_elbow_z', 'left_elbow_v', 'right_wrist_x', 'right_wrist_y', 'right_wrist_z', 'right_wrist_v', 'left_wrist_x', 'left_wrist_y', 'left_wrist_z', 'left_wrist_v', 'left_hip_x', 'left_hip_y', 'left_hip_z', 'left_hip_v', 'right_hip_x', 'right_hip_y', 'right_hip_z', 'right_hip_v']
Number of rows: 604 
Number of columns: 37

Labels: 
C    339
L    265
Name: label, dtype: int64

Missing values: False

Duplicate Rows : 0


In [15]:
# Standard Scaling of features
test_x = test_df.drop("label", axis = 1)
test_x = pd.DataFrame(sc.transform(test_x))

test_y = test_df["label"]

# # Converting prediction to categorical
test_y_cat = to_categorical(test_y)

In [16]:
test_set_results = []

for name, model in final_models.items():
    # Evaluate model
    predict_x = model.predict(test_x, verbose=False) 
    y_pred_class = np.argmax(predict_x, axis=1)
    y_test_class = np.argmax(test_y_cat, axis=1)

    cm = confusion_matrix(y_test_class, y_pred_class, labels=[0, 1])
    (p_score, r_score, f_score, _) = precision_recall_fscore_support(y_test_class, y_pred_class, labels=[0, 1])
    
    test_set_results.append(( name, round_up_metric_results(p_score), round_up_metric_results(r_score), round_up_metric_results(f_score), cm ))

test_set_results.sort(key=lambda k: k[1] + k[2] + k[3], reverse=True)
pd.DataFrame(test_set_results, columns=["Model", "Precision Score", "Recall Score", "F1 score", "Confusion Matrix"])

2022-11-25 15:34:28.174069: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-11-25 15:34:28.287586: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-11-25 15:34:28.423087: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-11-25 15:34:28.546100: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


,Model,Precision Score,Recall Score,F1 score,Confusion Matrix
0,7_layers,"[0.944, 1.0]","[1.0, 0.925]","[0.971, 0.961]","[[339, 0], [20, 245]]"
1,5_layers,"[0.926, 1.0]","[1.0, 0.898]","[0.962, 0.946]","[[339, 0], [27, 238]]"
2,7_layers_with_dropout,"[0.909, 0.963]","[0.973, 0.875]","[0.94, 0.917]","[[330, 9], [33, 232]]"
3,3_layers,"[0.896, 0.983]","[0.988, 0.853]","[0.94, 0.913]","[[335, 4], [39, 226]]"


## 5. Dumped Model

In [22]:
# Dump the best model to a pickle file
with open("./model/bicep_dp.pkl", "wb") as f:
    pickle.dump(final_models["7_layers"], f)

INFO:tensorflow:Assets written to: ram://4145d713-d810-484c-b518-b4ae694e4919/assets


In [23]:
# Dump final results
with open("./model/all_models.pkl", "wb") as f:
    pickle.dump(final_models, f)

INFO:tensorflow:Assets written to: ram://5ccb0e7f-b3f8-4602-9b3c-2ae89d1a2d69/assets
INFO:tensorflow:Assets written to: ram://5d2d95b4-ff82-487d-bd98-ba859e8eced0/assets
INFO:tensorflow:Assets written to: ram://557449b4-6368-4822-a75f-79675a055ab9/assets
INFO:tensorflow:Assets written to: ram://4857368e-b747-43dd-9b2d-6e986317f4b8/assets
